In [1]:
import requests
import json
import pandas as pd
from IPython.core.display import display

#This function checks whether or not the inputted college name exists in the database(Done by Xiao Ting)
def college_check(college_add,college_data):    
    for c in college_data['results']:
        if college_add.lower() in c['school']['name'].lower():
            return True
    return False

# This function calls to the College Scorecard API (Done by Emily)
def college_score(college_add):
    separator = '%20'
    tokens = college_add.split()
    name = separator.join(tokens)
    key = '1iVtjhPaaz47KC0WNWYPxklL3r8Fg7ss0mNxF56g'
    url = 'https://api.data.gov/ed/collegescorecard/v1/schools?school.name={}&api_key={}'.format(name,key)
    response = requests.get(url)
    college_data = response.json()
    return college_data

# This function prints out the results/information pertaining to each inputted college (Done by Xiao Ting)
def college_detail(college_list):
    detail_list = []
    for item in college_list:
        college_data = college_score(item)
        college_name = college_data['results'][0]['school']['name']
        reading = college_data['results'][0]['2014']['admissions']['sat_scores']['75th_percentile']['critical_reading']
        math = college_data['results'][0]['2014']['admissions']['sat_scores']['75th_percentile']['math']
        writing = college_data['results'][0]['2014']['admissions']['sat_scores']['75th_percentile']['writing']
        tuition = college_data['results'][0]['2014']['cost']['tuition']['in_state']
        school_url = college_data['results'][0]['school']['school_url']
        college_city = college_data['results'][0]['school']['city']
        college_state = college_data['results'][0]['school']['state']
        college_zip = college_data['results'][0]['school']['zip']
        location = college_city + ', ' + college_state
        geodata = google_geocode(location)
        coords =geodata['results'][0]['geometry']['location']
        weather = darksky_weather(coords)
        current = weather['currently']['temperature']
        dic = {'College Name': college_name, 'Tuition':tuition, 'SAT(R)':reading,'SAT(M)':math,
               'SAT(W)':writing, 'URL':school_url,'City': college_city, 'State': college_state, 
               'Zip': college_zip,'Temp': current}
        detail_list.append(dic)       
    detail_data = pd.DataFrame(detail_list,columns = ['College Name','Tuition','SAT(R)','SAT(M)','SAT(W)','URL','City','State','Zip','Temp'])
    return detail_data

# This function calls to the Google Maps API to get the location of each college (Done by Sophia)
def google_geocode(location): 
    params = {'address' : location}
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    response = requests.get(url, params = params)
    geodata = response.json()
    return geodata

# This function calls to the Dark Sky weather API to get the current weather conditions of each college (Done by Sophia)
def darksky_weather(coords):
    key = '8bbd2056a66915d251dd59abdf906bd2' 
    url='https://api.darksky.net/forecast/%s/%f,%f' % (key, coords['lat'], coords['lng'])
    response = requests.get(url)
    weather = response.json()
    return weather

#This function allows users to add colleges to their college list (Done by Emily)
def add_college(college_list,college_add):
    count = 0
    for college_name in college_list:
        if college_name == college_add:
            count = count + 1
            print('%s is already in the list!' %college_add)
    if count == 0:
            college_list.append(college_add)
    return college_list

#This function allows users to remove colleges to their college list (Done by Xiao Ting)
def remove_college(college_list,college_remove):
    count = 0
    for college_name in college_list:
        if college_name == college_remove:
            count = count + 1
            college_list.remove(college_remove)
    if count == 0:
        print('%s is not in the list!' %college_remove)
    return college_list  

# This function prompts users with a list of different options they can choose from to create their final college list
# (Done by Emily)
def list_menu(): 
    print('What you want to do NEXT?')
    print('A - Add college')
    print('R - Remove college')
    print('V - View colleges detail')
    print('Q - Quit the program')
    return


########################################## MAIN PROGRAM ##################################################
# This portion of the program prompts users to input a college name. The code is written so that user inputs that include lowercase and uppercase letters will still work with the program. 
# (Done by Xiao Ting)

college_list = []

while True:
    menu = list_menu()
    user_text = str(input('Please enter a letter option from the list (A, R, V or Q)): ')).upper()
    if user_text == 'A':
        college_add = str(input('Enter college name: ')).lower()
        college_data = college_score(college_add)
        result = college_check(college_add,college_data)
        if result == True: 
            college_list = add_college(college_list,college_add)
        if result == False:
            print('Sorry, this college can NOT be found in the data set.')
    elif user_text == 'R':
        college_remove = str(input('Which college would you like to remove? ')).lower()
        college_list = remove_college(college_list,college_remove)
    elif user_text == 'V':
        college_func = college_detail(college_list)
        display(college_func)
    elif user_text == 'Q':
        print('Thank you for using College Collider!')
        break


What you want to do NEXT?
A - Add college
R - Remove college
V - View colleges detail
Q - Quit the program
Please enter a letter option from the list (A, R, V or Q)): a
Enter college name: syracuse university
What you want to do NEXT?
A - Add college
R - Remove college
V - View colleges detail
Q - Quit the program
Please enter a letter option from the list (A, R, V or Q)): a
Enter college name: cornell university
What you want to do NEXT?
A - Add college
R - Remove college
V - View colleges detail
Q - Quit the program
Please enter a letter option from the list (A, R, V or Q)): v


,College Name,Tuition,SAT(R),SAT(M),SAT(W),URL,City,State,Zip,Temp
0,Syracuse University,41886,620.0,660.0,630.0,syr.edu/,Syracuse,NY,13244,54.49
1,Cornell University,47286,740.0,770.0,NaN,www.cornell.edu,Ithaca,NY,14853,53.90


What you want to do NEXT?
A - Add college
R - Remove college
V - View colleges detail
Q - Quit the program
Please enter a letter option from the list (A, R, V or Q)): q
Thank you for using College Collider!
